# Classification de la question en domaine

## 1. Pré-traitement textuel de la base de données

### A. Préparation du fichier

Nous importons le corpus de la composante conversationnelle Opus.

In [1]:
import pandas as pd

In [2]:
with open('../sources_opus/OpenSubtitles_fr.txt', 'r', encoding='utf-8') as f:
    data_opus = f.readlines() 

In [3]:
nb_lignes = len(data_opus)

In [4]:
from random import sample, seed
seed(1234)   # pour la reproductibilité des résultats
index_taked = sample(range(0,nb_lignes), 200)

In [5]:
lignes_opus = [data_opus[i] for i in index_taked]

In [6]:
df_opus = pd.DataFrame({'texte' : lignes_opus, 'domaine' : 0})

Nous importons la base créée dans le fchier .ipynb `Classification de la question en thématiques métier`.

In [7]:
df_faq = pd.read_pickle('df_classif_theme.pkl')

In [8]:
df_faq['domaine'] = 1
df_faq = df_faq[['texte', 'domaine']]

In [9]:
df_domaine = pd.concat([df_opus, df_faq])

Nous sauvegardons le DataFrame `df_domaine`.

In [10]:
df_domaine.to_pickle('df_classif_domaine.pkl')

## B. Nettoyage des données

Nous allons procéder à un nettoyage du tableau de données `df_domaine`.

In [11]:
import nltk
import string
import spacy
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import unidecode

Nous définissons un certain nombre de fonctions qui nous permettrons de nettoyer le corpus.

In [12]:
stemmer = SnowballStemmer('french')
nlp = spacy.load('fr_core_news_sm')

sw = nltk.corpus.stopwords.words('french')
sw += ['être', 'avoir']
sw.sort()

def lemmatise_text(text):
    lst_lematised = [token.lemma_ for token in nlp(text)] 
    return ' '.join(lst_lematised).lower()


def stem_text(text):
    lst_stemmerised = [stemmer.stem(token) for token in word_tokenize(text)]    
    return ' '.join(lst_stemmerised)


def substitute_punctuation(text):
    return ' '.join(text.replace("'", ' ').translate(str.maketrans('', '', string.punctuation)).split())


def substitute_special_char(text):
    return text.replace("«", "").replace("’", "").replace("•", "").replace("®", "")

Nous séparons le tableau `df_domaine` en un jeu d'apprentissage et un jeu de test. Nous conservons 70% des données dans le jeu d'apprentissage.

In [13]:
from sklearn.model_selection import train_test_split

df_domaine = pd.read_pickle('df_classif_domaine.pkl')
X_train, X_test, y_train, y_test = train_test_split(df_domaine['texte'], 
                                                    df_domaine['domaine'],
                                                    train_size=0.7,
                                                    random_state=5)

In [14]:
X_train_clean = (X_train.apply(lemmatise_text)
                        .apply(stem_text)
                        .apply(substitute_punctuation)
                        .apply(substitute_special_char)
                )

X_test_clean = (X_test.apply(lemmatise_text)
                      .apply(stem_text)
                      .apply(substitute_punctuation)
                      .apply(substitute_special_char)
               )

## 2. Test de différents modèles

Nous importons le classifieur TF-IDF.

In [15]:
from joblib import load
vectoriser_theme = load('vectorizer_classif_theme.joblib')

Nous allons maintenant procéder aux tests de différents classifieurs.

Nous entraînerons des modèles de classification appartenant à quelques familles d'algorithmes d'apprentissage automatique classique. L'objectif est de comparer non seulement les performances des différentes méthodes entre elles, mais aussi la performance d'une même méthode sur des représentations différentes du texte.

#### a. Régression Logistique

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

In [17]:
X_train_clean_vectorized_tfidf = vectoriser_theme.transform(X_train_clean)
X_test_clean_vectorized_tfidf = vectoriser_theme.transform(X_test_clean)

In [18]:
model_lr =  LogisticRegression(multi_class = 'multinomial', solver='lbfgs', max_iter=500).fit(X_train_clean_vectorized_tfidf, y_train)
predictions_valid = model_lr.predict(X_test_clean_vectorized_tfidf)
accuracy_score(y_test, predictions_valid)

0.8387096774193549

In [19]:
model_lr =  LogisticRegression(multi_class = 'multinomial', solver='newton-cg', max_iter=100, penalty="l2").fit(X_train_clean_vectorized_tfidf, y_train)
predictions_valid = model_lr.predict(X_test_clean_vectorized_tfidf)
accuracy_score(y_test, predictions_valid)

0.8387096774193549

#### b. SVM

In [20]:
from sklearn import svm

In [21]:
model_svm = svm.SVC(kernel='linear', C=10).fit(X_train_clean_vectorized_tfidf, y_train)
predictions_valid = model_svm.predict(X_test_clean_vectorized_tfidf)
accuracy_score(y_test, predictions_valid)

0.9032258064516129

## 3. Choix et sauvegarde

Bien que nous n'ayons pas laissé la trace de nos recherches, nous avons cherché, pour les deux modèles précédents, à ajuster au mieux leurs paramètres. Après ces différents tests, nous sauvegardons le modèle associé au classifieur qui nous fournit la meilleure accuracy.

In [22]:
from joblib import dump
dump(model_svm, 'model_classif_domaine.joblib')

['model_classif_domaine.joblib']

## 4. Evaluation quantitative

Afin d'évaluer de manière quantitative le modèle sélectionné, nous avons créé un jeu de questions - domaine. Chacune de ces questions est associé au domaine correspondant (1 correspond au domaine métier, 0 ne correspond pas). Les questions sont un mélange de celles de la FAQ ainsi que de celles qui ont été imaginées par nos soins. L'objectif étant de comparer la prédiction du modèle choisi au domaine attendu.

In [23]:
df_test_quanti = pd.read_csv('jeu_test_domaine.csv', ";")
df_test_quanti.head()

,question,domaine
0,Quels sont les jours d’arrivée ?,1
1,Comment évaluer le confort de mon domaine et d...,1
2,Quels sont les services et activités compris d...,1
3,Comment réserver mes activités ?,1
4,Où trouver le plan du domaine ?,1


Nous appliquons le même traitement que précédemment à ce jeu de données `df_test_quanti`. Nous séparons la variable d'intérêt de la variable explicative.

In [24]:
X_test_quanti = df_test_quanti.question
y_test_quanti = df_test_quanti.domaine

In [25]:
X_test_quanti = (X_test_quanti.apply(lemmatise_text)
                              .apply(stem_text)
                              .apply(substitute_punctuation)
                              .apply(substitute_special_char)
               )

In [27]:
X_test_quanti_vectorized_tfidf = vectoriser_theme.transform(X_test_quanti)

In [30]:
pred = model_svm.predict(X_test_quanti_vectorized_tfidf)

In [31]:
accuracy_score(y_test_quanti, pred)

0.8294117647058824

In [32]:
f1_score(y_test_quanti, pred)

0.8667687595712097

In [33]:
precision_score(y_test_quanti, pred)

0.9625850340136054

In [34]:
recall_score(y_test_quanti, pred)

0.7883008356545961

Les résultats obtenus sont satisfaisants puisque l'introduction de nouvelles questions ne perturbent pas le modèle. Le taux de nouvelles questions bien classées est de près de 83%.